# 🧩 Chapter 12: Clustering for Segmentation

Unsupervised learning (clustering) allows us to group points without labels. We explore:
1.  **K-Means**: Partitioning points into `k` groups (good for known object counts).
2.  **DBSCAN**: Density-based clustering (good for unknown counts and noise).

**Objectives:**
*   Segment a scene of planes using K-Means.
*   Segment a kitchen utilizing DBSCAN for object detection.

In [ ]:
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans, DBSCAN

## 1. K-Means Clustering

We have a scene with multiple planes. We want to separate them.

In [ ]:
# Load data
pcd = o3d.io.read_point_cloud("../DATA/KME_planes.xyz")
print(pcd)
o3d.visualization.draw_geometries([pcd], window_name="Input Data")

# Pre-processing (downsample if too large)
pcd_down = pcd.voxel_down_sample(voxel_size=0.05)
points = np.asarray(pcd_down.points)

# Run K-Means
k = 3 # We expect 3 planes
kmeans = KMeans(n_clusters=k, n_init=10).fit(points)
labels = kmeans.labels_

# Colorize
colors = plt.get_cmap("tab10")(labels / (k if k > 0 else 1))
pcd_down.colors = o3d.utility.Vector3dVector(colors[:, :3])

o3d.visualization.draw_geometries([pcd_down], window_name="K-Means Result")

## 2. DBSCAN Clustering

DBSCAN works on density. It connects points that are close together. It doesn't need to know `k` beforehand.

In [ ]:
# Load Kitchen Data
pcd_kitchen = o3d.io.read_point_cloud("../DATA/TLS_kitchen_sample.ply")
o3d.visualization.draw_geometries([pcd_kitchen], window_name="Kitchen Input")

# Run Open3D's optimized DBSCAN
print("Running DBSCAN...")
with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Debug) as cm:
    labels = np.array(pcd_kitchen.cluster_dbscan(eps=0.05, min_points=20, print_progress=True))

max_label = labels.max()
print(f"Found {max_label + 1} clusters.")

# Colorize
colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
colors[labels < 0] = 0 # Noise is black
pcd_kitchen.colors = o3d.utility.Vector3dVector(colors[:, :3])

o3d.visualization.draw_geometries([pcd_kitchen], window_name="Kitchen DBSCAN")